In [1]:
import requests
import pandas as pd
import time 

from league_summary_helper_functions import *

import json
# Load the configuration from config.json
with open('config.json') as config_file:
    config_data = json.load(config_file)

# I. Load Scores via ESPN API

In [34]:
#Load ESPN credentials
espn_cookies = {"swid": config_data['espn']['swid'],
                "espn_s2": config_data['espn']['espn_s2']}
league_id = config_data['espn']['league_id']
owner_dict = config_data['espn']['owner_dict']
year = 2023 #Change this to the current year

In [35]:
league_data = load_league(year, league_id, espn_cookies)
week = league_data['scoringPeriodId']-1
standings_df = load_records(league_data)
weekly_df = load_weekly_stats(year, league_id, espn_cookies, week)
schedule_df = load_schedule(year, league_id, espn_cookies, week)

In [39]:
standings_df

,team_id,owner,team_name,wins,losses,points_for,points_against,lowest_scoring_team
0,1,Tom Zhang,Early Celebration,1,0,132.32,79.24,0
10,12,John Choi,HOS 4SALE,1,0,121.86,76.82,0
6,7,Scott Lin,ME OKONKWO,1,0,91.34,70.96,0
1,2,Francis Jin,MackDarnolds chicken nuggets,1,0,90.34,87.80,0
7,9,John Qian,The Real Artist,1,0,85.04,74.00,0
13,15,Kyle O'Meara,Holistically Most Athletic,1,0,80.54,80.28,0
12,14,Andrew Kim,King Drew,1,0,75.26,74.16,0
5,6,Dan Jiang,Good Things Bad People,0,1,87.80,90.34,0
4,5,John Dong,Big Cheese,0,1,80.28,80.54,0
2,3,Peter Li,LI JINGPING,0,1,79.24,132.32,0


In [41]:
matchup_df = transform_weekly(weekly_df, schedule_df)
updated_standings = iterate_weeks_espn(year, week, standings_df, league_id, espn_cookies)
updated_standings.rename(columns={'team_id':'teamId'}, inplace=True)
matchup_df = updated_standings[['teamId', 'team_name']].merge(matchup_df, on='teamId', how='left')
matchup_df

Completed week: 1


,teamId,team_name,matchup_id,owner,total_points,QB,QB_PTS,RB1,RB1_PTS,RB2,...,WR2,WR2_PTS,TE,TE_PTS,Flex,Flex_PTS,D/ST,D/ST_PTS,K,K_PTS
0,1,Early Celebration,2,Tom Zhang,132.32,Justin Herbert,20.92,Miles Sanders,9.8,J.K. Dobbins,...,Stefon Diggs,21.2,Mark Andrews,0.0,Jahan Dotson,6.5,Cowboys D/ST,35.0,Jason Sanders,13.0
1,12,HOS 4SALE,6,John Choi,121.86,Daniel Jones,4.46,Javonte Williams,7.7,Rhamondre Stevenson,...,Tyreek Hill,39.0,Darren Waller,5.1,Michael Thomas,8.6,Commanders D/ST,12.0,Tyler Bass,13.0
2,7,ME OKONKWO,4,Scott Lin,91.34,Trevor Lawrence,17.74,Tony Pollard,21.2,Jahmyr Gibbs,...,Terry McLaurin,4.1,Dalton Schultz,1.4,Dameon Pierce,5.7,Jaguars D/ST,13.0,Younghoe Koo,7.0
3,2,MackDarnolds chicken nuggets,5,Francis Jin,90.34,Josh Allen,9.04,Austin Ekeler,24.4,Christian McCaffrey,...,Adam Thielen,2.2,Tyler Higbee,6.4,Devin Singletary,1.5,Panthers D/ST,6.0,Dustin Hopkins,12.0
4,9,The Real Artist,1,John Qian,85.04,Patrick Mahomes,19.54,Travis Etienne Jr.,18.9,Khalil Herbert,...,Mike Williams,6.5,Dallas Goedert,0.0,David Montgomery,13.4,Steelers D/ST,3.0,Justin Tucker,5.0
5,15,Holistically Most Athletic,7,Kyle O'Meara,80.54,Justin Fields,14.54,Joe Mixon,8.8,Dalvin Cook,...,CeeDee Lamb,9.7,Kyle Pitts,5.4,Odell Beckham Jr.,4.7,Saints D/ST,12.0,Harrison Butker,8.0
6,14,King Drew,3,Andrew Kim,75.26,Kirk Cousins,16.46,Josh Jacobs,8.1,Kenneth Walker III,...,Jaylen Waddle,9.8,Evan Engram,7.4,Chris Godwin,7.6,NaN,NaN,Daniel Carlson,5.0
7,6,Good Things Bad People,5,Dan Jiang,87.80,Jalen Hurts,12.50,Aaron Jones,25.7,Rashaad Penny,...,Amari Cooper,5.2,Noah Gray,4.6,Allen Lazard,5.6,Eagles D/ST,11.0,Cameron Dicker,12.0
8,5,Big Cheese,7,John Dong,80.28,Joe Burrow,3.18,Derrick Henry,12.9,Samaje Perine,...,Tyler Lockett,2.0,Chigoziem Okonkwo,0.0,Tyler Allgeier,22.9,Broncos D/ST,5.0,Eddy Pineiro,5.0
9,3,LI JINGPING,2,Peter Li,79.24,Tua Tagovailoa,27.14,Bijan Robinson,17.3,James Cook,...,Diontae Johnson,6.3,Dalton Kincaid,4.6,Drake London,0.0,Seahawks D/ST,-2.0,Brandon McManus,8.0


In [5]:
HP_Owner, HP_Player, HP_Score = highest_scoring_player_espn(weekly_df)
HT_Owner, HT_Score = highest_scoring_team_espn(weekly_df)

print(f'High Scoring Player: {HP_Owner}: {HP_Player}-{HP_Score}')
print(f'High Scoring Team: {HT_Owner} ({HT_Score})')

High Scoring Player: John Choi: Tyreek Hill-39.0
High Scoring Team: Tom Zhang (132.32)


# II. Create GPT Summary Email

In [42]:
import openai
openai.api_key = config_data['openai']['api_key']
from tabulate import tabulate
scoring_tabulate = tabulate(matchup_df.drop(columns = 'teamId'), headers='keys', tablefmt='plain', showindex=False)
standings_tabulate = tabulate(updated_standings, headers='keys', tablefmt='plain', showindex=False)

In [43]:
print(scoring_tabulate)

team_name                       matchup_id  owner            total_points  QB                 QB_PTS  RB1                   RB1_PTS  RB2                    RB2_PTS  WR1                  WR1_PTS  WR2                    WR2_PTS  TE                   TE_PTS  Flex                 Flex_PTS  D/ST               D/ST_PTS  K                  K_PTS
Early Celebration                        2  Tom Zhang              132.32  Justin Herbert      20.92  Miles Sanders             9.8  J.K. Dobbins              10.7  Chris Olave             15.2  Stefon Diggs              21.2  Mark Andrews            0    Jahan Dotson              6.5  Cowboys D/ST             35  Jason Sanders         13
HOS 4SALE                                6  John Choi              121.86  Daniel Jones         4.46  Javonte Williams          7.7  Rhamondre Stevenson       11.9  Calvin Ridley           20.1  Tyreek Hill               39    Darren Waller           5.1  Michael Thomas            8.6  Commanders D/ST          12  Ty

In [44]:
input_prompt = f"Week:{week}, \nScores Table: \n\n{scoring_tabulate}\n\nStandings Table: \n\n{standings_tabulate}\n\nSummary:"

response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo-16k",
  messages=[
    {
      "role": "system",
      "content": "You are an AI Fantasy Football commissioner tasked with writing a weekly summary to your league mates recapping the latest week of Fantasy Football.\n\nI will provide you a weekly scores table with each matchup (rows with the same matchup_id played each other and the total_points determines the winner), the owners, their players and what they scored. There will also be a standings table with everyone's records. \nUsing this information, write an email recapping the league in the style of Bill Simmons. Make sure to roast of the team with lowest total points \nMake the tone funny, light-hearted and slightly sarcastic "
    },
    {
      "role": "user",
      "content": input_prompt
    }
  ],
  temperature=1,
  max_tokens=4000,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

print(response['choices'][0]['message']['content'])

Hey there, fantasy fanatics!

Week 1 of our league is officially in the books, and boy, do we have some early winners and losers. But before we dive into the nitty-gritty, let's take a moment to give a shoutout to the team with the lowest total points of the week - KICKERS AND DEFENSE. Richard Liang, my friend, you set the bar nice and low with a whopping 70.96 points. Hey, at least you grabbed the title of "lowest-scoring team" right out of the gate. It's all about setting goals, right?

Now, onto the teams that actually managed to score some points and win their matchups. Leading the pack is Tom Zhang and his team, Early Celebration, with an impressive 132.32 points. Tom's players came to play, especially Stefon Diggs of the Bills, who snagged a cool 21.2 points. Tom, I hope you saved some champagne for the end of the season because your team is popping off already.

Right on Tom's heels is John Choi and HOS 4SALE with 121.86 points. John's team was helped out by Tyreek Hill, who abs

# III. Send G-Mail

In [21]:
owners_dict = {
    'Peter': 'peteryushunli@gmail.com',
    'Richard': 'Richardliang2015@gmail.com',
    'Johnny': 'choi.john91@gmail.com',
    #'Henry': 'henryzhang21@gmail.com',
    'Kyle': 'komearadoc@gmail.com',
    'Scott': 'lin.scott.bhs@gmail.com',
    'JD': 'johndong2015@u.northwestern.edu',
    #'David': 'dchoi26273@gmail.com',
    'Tom': 'tzhangbhs@gmail.com',
    'Kai': 'Kaishen2@gmail.com',
    'Dan': 'jiangd93@gmail.com',
    'Andrew': 'Andydozntcare@gmail.com',
    'Justin': "jchen49@gmail.com",
    'Richie' : 'richiekayy@gmail.com',
    'John Qian': 'Ziangqian@gmail.com',
    'Francis': 'francis0jin@gmail.com'
}
len(owners_dict.values())

14

In [48]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# Load the user name and password from yaml file
user, password = config_data['gmail']['GMAIL_USER'], config_data['gmail']['GMAIL_PW']

# Initialize the server
s = smtplib.SMTP('smtp.gmail.com', 587)
s.starttls()
s.login(user, password)

#Format the tables
scoring_tabulate = tabulate(matchup_df, headers='keys', tablefmt='html', showindex=False)
standings_tabulate = tabulate(updated_standings, headers='keys', tablefmt='html', showindex=False)

# Define message
subject = f"Weekly Fagudas FF Recap: Week {week}"
summary = response['choices'][0]['message']['content']
summary_html = summary.replace('\n', '<br>')
body = f"""\
<html>
  <head></head>
  <body>
    <br>{summary_html}</br>
    <br>
    <hr>
    <p><b>Highest Scoring Team</b>: {HT_Owner}: {HT_Score} points</p>
    <p><b>Highest Scoring Player</b>: {HP_Owner} - {HP_Player}: {HP_Score} Points</p>
    <br>
    {standings}
  </body>
</html>
"""

recipients = list(owners_dict.values())
#recipients = ['lithal06@yahoo.com']
sender = "peteryushunli@gmail.com"

msg = MIMEMultipart("alternative")
msg["From"] = sender
msg["To"] = ", ".join(recipients)
msg["Subject"] = subject

# Attach the HTML body
msg.attach(MIMEText(body, 'html'))

# Send email
s.sendmail(sender, recipients, msg.as_string())

# Terminate the session
s.quit()

(221,
 b'2.0.0 closing connection l2-20020ac84cc2000000b00403bf34266csm211064qtv.30 - gsmtp')